# Wpowadzenie do deep learning w bibliotece Flux.jl

## Przykład

Aby w  zrozumieć sposób pracy z Fluxem warto rozpatrzeć prosty przykład. Zajmiemy się przetwarzaniem języka naturalnego - zbudujemy model zdolny do generowania składnej wypowiedzi w języku polskim.

Wyjściowe założenie jest takie, że wytrenujemy sieć neuronową, która będzie estymowała prawdopodobieństwo wystąpienia danego znaku w ciągu na podstawie poprzedzających go znaków w sekwencji ([<b>Character-Level Language Model</b>](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)).

Zbiórem na którym będziemy pracowali jest <i>W poszukiwaniu straconego czasu</i> Marcela Prousta. 

[![](https://upload.wikimedia.org/wikipedia/commons/b/b8/Marcel_Proust_1895.jpg)](https://pl.wikipedia.org/wiki/Marcel_Proust)

>(...) matka widząc, że mi jest zimno, namówiła mnie, abym się napił wbrew zwyczajowi trochę herbaty. Odmówiłem zrazu; potem, nie wiem czemu, namyśliłem się. Posłała po owe krótkie i pulchne ciasteczka zwane magdalenkami, które wyglądają jak odlane w prążkowanej skorupie muszli. I niebawem (...) machinalnie podniosłem do ust łyżeczkę herbaty, w której rozmoczyłem kawałek magdalenki. Ale w tej samej chwili, kiedy łyk pomieszany z okruchami ciasta dotknął mego podniebienia, zadrżałem, czując, że się we mnie dzieje coś niezwykłego. Owładnęła mną rozkoszna słodycz (...). Sprawiła, że w jednej chwili koleje życia stały mi się obojętne, klęski jako błahe, krótkość złudna (...). Cofam się myślą do chwili, w której wypiłem pierwszą łyżeczkę herbaty (...). I nagle wspomnienie zjawiło mi się. Ten smak to była magdalenka cioci Leonii.(...)

Zanim jednak zaczniemy wprowadźmy odrobinę teorii stojącej za tym zagadnieniem:

### Rekurencyjne sieci neuronowe (Recurrent neural networks)

- Charakterystyczną cechą tego typu sieci jest to, że pozwalają one na istnienie wewnątrz grafu cykli skierowanych.
- Oznacza to, że informacja wewnątrz takiej sieci nie musi płynąć tylko w jednym kierunku - neurony leżące na tej samej warstwie także mogą przesyłać sobie wzajemnie dane:

[![](http://karpathy.github.io/assets/rnn/diags.jpeg)](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

Dzięki tej właściwości RNN doskonale nadają się do budowy interesującego nas modelu: 

[![](http://karpathy.github.io/assets/rnn/charseq.jpeg)](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

### Long short-term memory

Problemem na który można natrafić w przypadku korzystania z RNN jest pamięć takiej sieci. Gdy odległość pomiędzy aktualnym a poprzedzającymi go węzłami, które niosą za sobą kluczową informację jest niewielka, sieć jest w stanie efektywnie je wykorzystać:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-shorttermdepdencies.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Problem się pojawia gdy ta odległość jest duża - wtedy kluczowe informacje po prostu znikają w szumie:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-longtermdependencies.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Wtedy też, warto zastosować sieć LSTM, która ze względu na swoją architekturę jest w stanie odpowiednio filtrować informację i wykorzystawać je nawet wtedy, gdy ich źródło jest znacznie oddalone od aktualnego neuronu:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

### Alternatywy

Zamiast bazowych sieci rekurencyjnych lub sieci LSTM (i ich [modyfikacji](https://en.wikipedia.org/wiki/Long_short-term_memory)) można zastosować różne alternatywy, np. sieci <b>Gated Recurrent Unit<b> (GRU):
    
[![](https://upload.wikimedia.org/wikipedia/commons/5/5f/Gated_Recurrent_Unit.svg)](https://en.wikipedia.org/wiki/Gated_recurrent_unit)

Lub inne modele skonstruowane do rozwiązywania specyficznych problemów, np. [uczenia na szeregach czasowych.](https://github.com/sdobber/FluxArchitectures)

Przejdźmy teraz do implementowania modelu za pomocą Fluxa:

### Implementacja

In [1]:
using Flux
using Flux: onehot, argmax, chunk, batchseq, throttle, crossentropy
using StatsBase: wsample
using Base.Iterators: partition
using BSON
using CUDA

In [2]:
use_cuda = true

true

In [3]:
 if use_cuda && CUDA.functional()
    device = gpu
    @info "Training on GPU"
else
    device = cpu
    @info "Training on CPU"
end

┌ Info: Training on GPU
└ @ Main In[3]:3


Pierwszym krokiem jest oczywiście odpowiednie przygotowanie danych na których będziemy pracowali:

In [4]:
isfile("w_poszukiwaniu.txt") ||
        download("https://raw.githubusercontent.com/bartoszpankratz/221660-0553-Aproksymacja/master/6.%20Sieci%20Rekurencyjne/w_poszukiwaniu.txt","w_poszukiwaniu.txt")

true

In [5]:
text = collect(read("w_poszukiwaniu.txt",String));
alphabet = [unique(text)..., '_'];

Następnie kodujemy zmienne jakościowe:

In [6]:
text = map(ch -> onehot(ch, alphabet), text);
stop = onehot('_', alphabet);

In [7]:
N = length(alphabet);
seqlen = 100;
batch_size = 32;

In [8]:
Xs = collect(partition(batchseq(chunk(text, batch_size), stop), seqlen)) |> device;
Ys = collect(partition(batchseq(chunk(text[2:end], batch_size), stop), seqlen)) |> device;

In [9]:
m = Chain(
  LSTM(N, 128),
  LSTM(128, 256),
  LSTM(256, 128),
  Dense(128, N),
  softmax) |> device

function loss(xs, ys, ϵ = 1.0f-32)
  l = sum(crossentropy.(broadcast(x -> m(x) .+ ϵ, xs), ys))
  Flux.reset!(m)
  return l
end

opt = ADAM(0.001)


function sample(m, alphabet, len; temp = 1)
    model = cpu(m)
    Flux.reset!(model)
    buf = IOBuffer()
    c = rand(alphabet)
    for i = 1:len
        write(buf, c)
        c = wsample(alphabet, model(onehot(c, alphabet)))
      end
    return String(take!(buf))
end

evalcb = function ()
    @show loss(Xs[5], Ys[5])
    println(sample(m, alphabet, 100))
end

#6 (generic function with 1 method)

In [10]:
loss(Xs[5], Ys[5])

481.29572f0

In [11]:
sample(m, alphabet, 50)

"ïM4qQaôżu\n\n.Nt„H-èâ!ż6L)UhI:iàłBUrAśVDĘŹw;ŻDiŁfVóR"

In [12]:
@info("Beginning training loop...")
best_ls = Inf
last_improvement = 0
for epoch = 1:20
    @info "Epoch: $epoch"
    global best_ls, last_improvement
    Flux.train!(loss, params(m), zip(Xs, Ys), opt, cb=throttle(evalcb, 240))
    ls = loss(Xs[5], Ys[5])
    if ls <= best_ls
        @info "New best result: $ls"
        BSON.@save "char_model.bson" m
        best_ls = ls
        last_improvement = epoch
    end
    if epoch - last_improvement >= 5
        @warn(" -> We're calling this converged.")
        break
    end
end

loss(Xs[5], Ys[5]) = 479.3394f0
«V3x»2»9ÊoĘUF4XrłCkŚâRT﻿Ż5eżód ö1»El.EWYc…6-Ó;1c»çkhuMŹq„kôB…ywgÊA_e?fO”C—ïî;7!/óÊW)ùłS«lz½q­(ŹEIùôZ
loss(Xs[5], Ys[5]) = 325.0689f0
/mk ldao iijksna ,dguwker io ą .ap,i  a uiioojw eaor,yk pmii mike yrs azlit rol  amudzyynrgbejagacau
loss(Xs[5], Ys[5]) = 324.77625f0
vieaasm  awniwwz,ę  z,iwśIdśamaioor e so dgey lJeo wioiał o  sgb ditdeo  erąaoriIwCenlnr s,cazs ęnne
loss(Xs[5], Ys[5]) = 324.93393f0
św ttkoaóekzuo,  sonzbczt,esse o  ciśbwuznyaagnmmnioezsew.inakP.l,rten łęyzeilsęcs i aera.y gj ireze
loss(Xs[5], Ys[5]) = 324.8353f0
mąaa    itenł kz n obdydmt”  ókłknyO 
yzllye  sesnojó a Ckbw z nęaacu  szr-s wśęss 
ręm.imłciwcyewPt
loss(Xs[5], Ys[5]) = 325.2941f0
A   eduZą  tęćzbsimkni cohwmnałgsdmilmeerndrsykądcseiwaiou jgitzhikoy skcrołodńepr ujjnimwern seeók 
loss(Xs[5], Ys[5]) = 324.93268f0
﻿zcwia oo ż cwkzaw  wecla k td
o.pątk ożuiwzjjor,ajOkbjałriktpz an wazaktgułtkwwtd .yoe yi ćBe jrzu 
loss(Xs[5], Ys[5]) = 324.97333f0
.dząlrrabrcidzzżamteo iw     yrjw

┌ Info: Beginning training loop...
└ @ Main In[12]:1
┌ Info: Epoch: 1
└ @ Main In[12]:5
┌ Info: New best result: 324.94272
└ @ Main In[12]:10
┌ Info: Epoch: 2
└ @ Main In[12]:5
┌ Info: New best result: 324.94162
└ @ Main In[12]:10
┌ Info: Epoch: 3
└ @ Main In[12]:5
┌ Info: New best result: 324.94025
└ @ Main In[12]:10
┌ Info: Epoch: 4
└ @ Main In[12]:5
┌ Info: New best result: 324.93622
└ @ Main In[12]:10
┌ Info: Epoch: 5
└ @ Main In[12]:5
┌ Info: New best result: 324.93616
└ @ Main In[12]:10
┌ Info: Epoch: 6
└ @ Main In[12]:5
┌ Info: Epoch: 7
└ @ Main In[12]:5
┌ Info: New best result: 293.82513
└ @ Main In[12]:10
┌ Info: Epoch: 8
└ @ Main In[12]:5
┌ Info: New best result: 246.96013
└ @ Main In[12]:10
┌ Info: Epoch: 9
└ @ Main In[12]:5
┌ Info: New best result: 238.11874
└ @ Main In[12]:10
┌ Info: Epoch: 10
└ @ Main In[12]:5
┌ Info: New best result: 236.81682
└ @ Main In[12]:10
┌ Info: Epoch: 11
└ @ Main In[12]:5
┌ Info: New best result: 236.71828
└ @ Main In[12]:10
┌ Info: Epoch: 12
└ 

In [13]:
BSON.@load "char_model.bson" m

In [14]:
sample(m, alphabet, 50)

"ł spaje  zażar. Pacze kielberdziały najczarzurinie"

## Dodatkowa praca domowa

Zmodyfikuj kod tak, aby poprawić jakość generowanego tekstu <b>(5 punktów)</b>.